# predictions

> We define all the usefull functions to predict the score from a given sequence

In [ ]:
#| default_exp predictions

In [ ]:
#| export
import pickle
import sklearn
import dgrec
import numpy as np
import pandas as pd
from dgrec.example_data import get_example_data_dir
import os


In [ ]:
# data_path=get_example_data_dir()
# model_name='model_mms_2024_02_14.pickle'
# model_path=os.path.join(data_path,model_name)
# model=pickle.load(open('/home/prochett/DGRec/dgrec/example_data/model_mms_2024_02_14.pickle',"rb"))

In [ ]:
# | export

def score(TR_seq:str #A string of the TR DNA sequence
,model):

    """Calculates the predicted score of a given TR sequence (1 = perfect TR and 0 = crappy TR)"""
    encoded_TR=dgrec.encoding.encode_tr_list([TR_seq])
    score=np.round(model.predict_proba([encoded_TR[0]])[:,1],decimals=2).item()
    return score

In [ ]:
# TR_bad='TTAGCGAATGGCGAAATTCGTAAACGCCCTCTGATCGAAACCAACGGCGAAACGGGTGAGATCGTGTGGG'
# print('TR bad score =',score(TR_bad,model))
# TR_good='AAATGATCGCCAAATCTGAACAGGAAATTGGCAAAGCAACCGCTAAATACTTTTTCTACTCAAACATTAT'
# print('TR good score =',score(TR_good,model))

100%|██████████| 1/1 [00:00<00:00,  3.91it/s]


TR bad score = 0.13


100%|██████████| 1/1 [00:00<00:00,  3.96it/s]

TR good score = 0.96


In [ ]:
# | export
def score_list(TR_seq_list:list, #A list of strings of TRs DNA sequences
TR_name_list:list, #A list of strings of TRs names
model):
    """Calculates the score for every TR in the list and returns them in a dataframe format"""

    encoded_TR=dgrec.encoding.encode_tr_list(TR_seq_list)
    score=np.round(model.predict_proba(encoded_TR)[:,1],decimals=2)
    score_df=pd.DataFrame({
        'TR_Name':TR_name_list,
        'TR_Seq':TR_seq_list,
        'TR_Score':score
    })
    return score_df

In [ ]:
# TR_bad=[
#     'TTAGCGAATGGCGAAATTCGTAAACGCCCTCTGATCGAAACCAACGGCGAAACGGGTGAGATCGTGTGGG',
#     'AAACGCCCTCTGATCGAAACCAACGGCGAAACGGGTGAGATCGTGTGGGACAAAGGTCGTGATTTCGCTA',
#     'GGTTTCTCTAAGGAGTCCATTCTGCCGAAGCGCAACTCCGACAAGCTGATCGCGCGTAAGAAGGACTGGG',
#     'CAAGCTGATCGCGCGTAAGAAGGACTGGGATCCGAAGAAGTACGGTGGCTTCGATTCTCCGACCGTGGCG',
#     'ACCCGATTGACTTCCTCGAGGCGAAGGGGTACAAGGAGGTGAAGAAGGATCTGATTATCAAGCTGCCGAA',
#     'AGTACTCCCTGTTCGAGCTGGAGAATGGTCGTAAGCGTATGCTGGCGTCTGCGGGTGAGCTGCAGAAGGG',
#     'CAGCACAAGCACTACCTGGACGAGATTATTGAGCAGATTTCTGAGTTTTCTAAGCGCGTGATTCTGGCGG',
#     'ACGCGAATCTGGATAAGGTCCTGTCTGCCTACAATAAGCACCGTGATAAGCCGATCCGTGAGCAGGCGGA',   
# ]

# score_list(TR_bad,['TR_bad_'+str(k) for k in range (1,9)],model)


100%|██████████| 8/8 [00:01<00:00,  4.06it/s]


,TR_Name,TR_Seq,TR_Score
0,TR_bad_1,TTAGCGAATGGCGAAATTCGTAAACGCCCTCTGATCGAAACCAACG...,0.13
1,TR_bad_2,AAACGCCCTCTGATCGAAACCAACGGCGAAACGGGTGAGATCGTGT...,0.12
2,TR_bad_3,GGTTTCTCTAAGGAGTCCATTCTGCCGAAGCGCAACTCCGACAAGC...,0.01
3,TR_bad_4,CAAGCTGATCGCGCGTAAGAAGGACTGGGATCCGAAGAAGTACGGT...,0.02
4,TR_bad_5,ACCCGATTGACTTCCTCGAGGCGAAGGGGTACAAGGAGGTGAAGAA...,0.01
5,TR_bad_6,AGTACTCCCTGTTCGAGCTGGAGAATGGTCGTAAGCGTATGCTGGC...,0.03
6,TR_bad_7,CAGCACAAGCACTACCTGGACGAGATTATTGAGCAGATTTCTGAGT...,0.09
7,TR_bad_8,ACGCGAATCTGGATAAGGTCCTGTCTGCCTACAATAAGCACCGTGA...,0.05


In [ ]:
# TR_good=[
#     'AAATGATCGCCAAATCTGAACAGGAAATTGGCAAAGCAACCGCTAAATACTTTTTCTACTCAAACATTAT',
#     'TCAAACATTATGAATTTCTTCAAAACCGAAATCACCTTAGCGAATGGCGAAATTCGTAAACGCCCTCTGA',
#     'ATGCCTCAAGTAAACATCGTTAAAAAGACTGAGGTGCAGACTGGCGGTTTCTCTAAGGAGTCCATTCTGC',
#     'GGATCCGAAGAAGTACGGTGGCTTCGATTCTCCGACCGTGGCGTACTCTGTTCTGGTGGTCGCCAAGGTC',
#     'AGCGTATGCTGGCGTCTGCGGGTGAGCTGCAGAAGGGGAACGAGTTGGCCCTTCCGTCCAAGTACGTGAA',
#     'GCAGAAGGGGAACGAGTTGGCCCTTCCGTCCAAGTACGTGAACTTCCTGTACCTGGCCTCGCACTACGAG',
#     'CAGAAGCAGCTGTTCGTGGAGCAGCACAAGCACTACCTGGACGAGATTATTGAGCAGATTTCTGAGTTTT',
#     'CTAAGCGCGTGATTCTGGCGGACGCGAATCTGGATAAGGTCCTGTCTGCCTACAATAAGCACCGTGATAA'
#     ]

# score_list(TR_good,['TR_good_'+str(k) for k in range (1,9)],model)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:01<00:00,  4.03it/s]


,TR_Name,TR_Seq,TR_Score
0,TR_good_1,AAATGATCGCCAAATCTGAACAGGAAATTGGCAAAGCAACCGCTAA...,0.96
1,TR_good_2,TCAAACATTATGAATTTCTTCAAAACCGAAATCACCTTAGCGAATG...,0.95
2,TR_good_3,ATGCCTCAAGTAAACATCGTTAAAAAGACTGAGGTGCAGACTGGCG...,0.92
3,TR_good_4,GGATCCGAAGAAGTACGGTGGCTTCGATTCTCCGACCGTGGCGTAC...,0.89
4,TR_good_5,AGCGTATGCTGGCGTCTGCGGGTGAGCTGCAGAAGGGGAACGAGTT...,0.91
5,TR_good_6,GCAGAAGGGGAACGAGTTGGCCCTTCCGTCCAAGTACGTGAACTTC...,0.83
6,TR_good_7,CAGAAGCAGCTGTTCGTGGAGCAGCACAAGCACTACCTGGACGAGA...,0.90
7,TR_good_8,CTAAGCGCGTGATTCTGGCGGACGCGAATCTGGATAAGGTCCTGTC...,0.92
